Github repository is here: <https://github.com/ong8181/DASC3240>

# Reproducing the previous plots with ggplot2

## References

-   Data Analysis and Visualization in R for Ecologists: <https://datacarpentry.org/R-ecology-lesson>
-   Data manipulation: <https://datacarpentry.org/R-ecology-lesson/03-dplyr.html>
-   Data visualization with ggplot2: <https://datacarpentry.org/R-ecology-lesson/04-visualization-ggplot2.html>
-   Zenodo archive: <http://doi.org/10.5281/zenodo.3264888>
-   License (CC-BY4.0): <https://github.com/datacarpentry/R-ecology-lesson?tab=License-1-ov-file>
-   Magwene, P. M. "Introduction to ggplot2", Biology 304: Biological Data Analysis. (Accessed 25 Jan 2024). <https://bio304-class.github.io/bio304-book/introduction-to-ggplot2.html> (reused with permission from Prof. Magwene)

# Contents

1.  Plotting one variable: Histogram and density plot
2.  Plotting two variables: Barplot, boxplot, jitterplot, and violinplot
3.  Plotting two variables: Scatterplot
4.  Plotting two variables: Time series
5.  Visualization of high dimensional data
6.  Map
7.  Useful tips

# 1. Plotting one variable: Histrogram and density plot

In this hands-on, we will use `iris` dataset.

Reference: <https://bio304-class.github.io/bio304-book/introduction-to-ggplot2.html>

In [ ]:
# Load library
library(tidyverse); packageVersion("tidyverse")

In [ ]:
# Load iris
data(iris)

In [ ]:
# Histogram
ggplot(iris) + 
  geom_histogram(aes(x = Sepal.Width))
#> `stat_bin()` using `bins = 30`. Pick better value with `binwidth`.

In [ ]:
ggplot(iris) + 
  geom_density(aes(x = Sepal.Width, fill = Species), alpha=0.25)

# 2. Plotting two variables: Barplot, boxplot, jitterplot, and violinplot

## Barplot

In [ ]:
ggplot(iris, aes(x = Species)) + 
  geom_bar()

In [ ]:
iris %>%
   group_by(Species) %>%
   summarize(mean_Sepal.Width = mean(Sepal.Width)) %>%
   ggplot(aes(x = Species, y = mean_Sepal.Width)) + geom_bar(stat = "identity")

### Stacked barplot (community composition)

In [ ]:
# Create artificial data
d_com <- data.frame(Plot1 = c(10, 5, 5, 0),
                    Plot2 = c(8, 7, 5, 4),
                    Plot3 = c(5, 5, 5, 15),
                    Plot4 = c(0, 5, 8, 17))
rownames(d_com) = c("A", "B", "C", "D")

# Calculate the relative abundance
d_com <- apply(d_com, 2, function(x) x/sum(x))
d_com <- data.frame(d_com)

# Convert the rownames to column
d_com <- d_com %>% rownames_to_column(var = "species")

# Convert the data.frame to a longer format
d_com_long <- d_com %>% pivot_longer(cols = -species, names_to = "plot", values_to = "relative_abundance")

# Visualize using stacked barplots
d_com_long %>%
  ggplot(aes(x = plot, y = relative_abundance, fill = species)) +
  geom_bar(stat = "identity") +
  xlab("Plot") + ylab("Relative abundance")

## Boxplot

In [ ]:
ggplot(iris, aes(x = Species, y = Sepal.Width, color = Species)) +
   geom_boxplot()

## Violinplot

In [ ]:
ggplot(iris) + 
  geom_violin(aes(x = Species, y = Sepal.Width, color = Species, fill=Species), 
              alpha = 0.25)

## Jitterplot

In [ ]:
ggplot(iris, aes(x = Species, y = Sepal.Width, color = Species)) +
   geom_jitter(height = 0, width = 0.2)

## Jitterplot + boxplot

In [ ]:
ggplot(iris, aes(x = Species, y = Sepal.Width, color = Species)) +
   geom_boxplot(outlier.shape = NA) +
   geom_jitter(height = 0, width = 0.2, size = 2, alpha = 0.8)

# 3. Plotting two variables: Scatterplot

In [ ]:
# Standard scatterplot
ggplot(iris, aes(x = Sepal.Length, y = Sepal.Width, color = Species)) +
   geom_point()

In [ ]:
# Scatterplot with LOWESS fitting curve
ggplot(iris, aes(x = Sepal.Length, y = Sepal.Width, color = Species)) +
   geom_point() +
   stat_smooth()

In [ ]:
# Scatterplot with linear regression
ggplot(iris, aes(x = Sepal.Length, y = Sepal.Width, color = Species)) +
   geom_point() +
   stat_smooth(method = "lm")

In [ ]:
# Change colors
ggplot(iris, aes(x = Sepal.Length, y = Sepal.Width, color = Species)) +
   geom_point() +
   stat_smooth() +
   scale_color_manual(values = c("gray30", "red3", "royalblue"))

In [ ]:
# Use facet to show the data in separate panels
ggplot(iris, aes(x = Sepal.Length, y = Sepal.Width, color = Species)) +
   geom_point() +
   stat_smooth() +
   facet_wrap(~ Species)

# 4. Plotting two variables: Time series

As in the Python examples, we need to care about "date" variables. **lubridate** package is a great one to handle date variables.

In [ ]:
# Load library
library(tidyverse)
library(lubridate) # If not included in the tidyverse package

In [ ]:
# Load dataset from web
## see https://r-graph-gallery.com/line_chart_annotation.html
d <- read.table("https://raw.githubusercontent.com/holtzy/data_to_viz/master/Example_dataset/3_TwoNumOrdered.csv", header = T)
d$date <- ymd(d$date)

In [ ]:
# Generate time series plot using geom_line
d %>% 
  ggplot(aes(x = date, y = value)) +
    geom_line()

In [ ]:
# Transform y-axis to log-scale
d %>% 
  ggplot(aes(x = date, y = value)) +
    geom_line() +
    scale_y_log10()

# 5. Visualization of high dimensional data

Try dimension reduction methods using R. We will use the same data with what we used in the Python session.

In [ ]:
d <- read.csv("data/eDNA_copy_site_top.csv", row.names = 1)
head(d)

In [ ]:
# Remove meta-data
d_abn <- d %>% select(-c(site_name, lat_n, long_e))
head(d_abn)

### Principal Component Analysis (PCA)

In [ ]:
# Calculate eDNA community composition instead of performing standardization
d_comp <- t(apply(d_abn, 1, function(x) x/sum(x)))

# PCA using prcomp()
pca_res <- prcomp(d_comp)
str(pca_res)

In [ ]:
pca_res$x

In [ ]:
# Plot the results using ggplots
pca_res$x %>% as.data.frame %>% # data should be data.frame or tibble
   ggplot(aes(x = PC1, y = PC2)) +
      geom_point(aes(color = d$site_name))

### Principal Coordinate Analysis (or MDS)

In R, PCoA can be performed using `vegan::cmdscale()`.

In [ ]:
# Load library
library(vegan) # For community ecology analysis including NMDS

# Calculate distance matrix (or dissimilarity)
d_dist <- vegdist(d_comp, method = "bray")

# Do PCoA
pcoa_res <- cmdscale(d_dist, eig = TRUE)

In [ ]:
# Visualize the result using ggplots
pcoa_res$points %>% as.data.frame %>%
   ggplot(aes(x = V1, y = V2)) +
      geom_point(aes(color = d$site_name)) +
      xlab("PCoA axis 1") + ylab("PCoA axis 2")

### Nonmetric Dimensional Scaling (NMDS)

In [ ]:
# Load library
library(vegan)

# Set random seed
set.seed(1234) # Better to set a random seed as NMDS includes some random processes

# Do NMDS
nmds_res <- metaMDS(d_comp, # data
                     distance = "bray", # Use Bray-Curtis dissimilarity
                     k = 2) # The number of reduced dimensions

In [ ]:
# Visualize the result using ggplots
nmds_res$points %>% as.data.frame %>%
   ggplot(aes(x = MDS1, y = MDS2)) +
      geom_point(aes(color = d$site_name)) +
      xlab("Axis 1") + ylab("Axis 2")

### t-SNE

We will install [**Rtsne**](https://github.com/jkrijthe/Rtsne) package to perform t-SNE.

In [ ]:
# Installation
#install.packages("Rtsne")

# Load library
library(Rtsne)

In [ ]:
# Sets seed for reproducibility
set.seed(1234)

# Run t-SNE
tsne_out <- Rtsne(as.matrix(d_abn), dims = 2, perplexity = 30)
# dims = Output dimension
# perplexity = An index of how many neighbors (how far/near) we will consider (small value means it considers a local structure only)
str(tsne_out)

In [ ]:
tsne_out$Y %>% as.data.frame %>%
   ggplot(aes(x = V1, y = V2)) +
      geom_point(aes(color = d$site_name)) +
      xlab("Axis 1") + ylab("Axis 2")

# 6. Map

Draw maps using [`ggmap` package](https://github.com/dkahle/ggmap). We will use [Stadia Maps](https://stadiamaps.com/) data.

In [ ]:
# Load library
#devtools::install_github("dkahle/ggmap", ref = "tidyup")
library(ggmap); packageVersion("ggmap") # 4.0.0

In [ ]:
# Register Stadia Maps API key
## You should sign up Stadia Maps and get your own API
## API key is a key that is required for apps to communicate each other (R and Stadia Maps, in this case)
ggmap::register_stadiamaps("xxxxxxxxxxxxxxxxxxxxx")

# Map type names
map_types <- c("stamen_terrain", "stamen_toner", "stamen_toner_lite",
               "stamen_watercolor", "alidade_smooth", "alidade_smooth_dark",
               "outdoors", "stamen_terrain_background", "stamen_toner_background",
               "stamen_terrain_labels", "stamen_terrain_lines",
               "stamen_toner_labels", "stamen_toner_lines")

# Draw Hong Kong Map
hk_lonlat <- c(left = 113.8, right = 114.5, top = 22.6, bottom = 22.1)
hk_detail <- get_stadiamap(hk_lonlat, zoom = 10, maptype = map_types[1])

In [ ]:
ggmap(hk_detail)

# 7. Useful tips

## Axis label angle and position

In [ ]:
g1 <- ggplot(iris, aes(x = Species, y = Sepal.Width, color = Species)) +
   geom_boxplot(outlier.shape = NA) +
   geom_jitter(height = 0, width = 0.2, size = 2, alpha = 0.8)

g1 + theme(axis.text.x = element_text(angle = 90))

In [ ]:
g1 + theme(axis.text.x = element_text(angle = 90, vjust = 0))
g1 + theme(axis.text.x = element_text(angle = 90, vjust = 0.5))
g1 + theme(axis.text.x = element_text(angle = 90, vjust = 1))

## Change colors manually

In [ ]:
g1 + scale_color_manual(values = c("black", "red", "blue"))

## Change legend title

In [ ]:
g1 + scale_color_manual(name = "New legend title", values = c("black", "red", "blue"))

## Use scientific notation for large values

In [ ]:
# Define a custom function
# https://stackoverflow.com/questions/10762287/how-can-i-format-axis-labels-with-exponents-with-ggplot2-and-scales
label_10_to_power <- function (x) {
    ifelse(x == 0, "0", parse(text = gsub("[+]", "", gsub("e", 
        " %*% 10^", (scales::scientific_format())(x)))))
  }

# Load data again
## see https://r-graph-gallery.com/line_chart_annotation.html
d <- read.table("https://raw.githubusercontent.com/holtzy/data_to_viz/master/Example_dataset/3_TwoNumOrdered.csv", header = T)
d$date <- ymd(d$date)

d %>% 
  ggplot(aes(x = date, y = value)) +
    geom_line() + scale_y_continuous(label = label_10_to_power)